In [1]:
import h5py
import numpy as np
import random
import pdb
from generator_helper_functions import create_user_maxtrix, create_user_ids, latent_user_input_layer

f = h5py.File('../data/Full_NF.h5', 'r')
dset = f['dataset_1']
dset = dset[:50000000]
print("Data loaded from file")

dset_indicies = set(range(len(dset)))
train_num = int(.9 * len(dset))
print("Training Samples: ", train_num, "| Validation Samples: ", len(dset) - train_num)

user_ids = create_user_ids(dset)
train_indicies = random.sample(dset_indicies, train_num)
validation_indicies = dset_indicies.difference(train_indicies)
training_data = np.array([dset[i] for i in train_indicies])
del train_indicies
validation_data = np.array([dset[i] for i in validation_indicies])
del validation_indicies
print("Training and Validation datasets separated")
#del dset

/home/cody/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Data loaded from file
Training Samples:  45000000 | Validation Samples:  5000000
Training and Validation datasets separated


In [2]:
user_ids = create_user_ids(dset)
k = 50

In [3]:
#train_indices[0]

In [4]:
del dset_indicies
#del train_indicies
#del validation_indicies
del dset

In [5]:
user_matrix = create_user_maxtrix(training_data, user_ids, k_n=k)

In [6]:
import pickle

In [7]:
with open('../data/user_matrix.pickle', 'wb') as handle:
    pickle.dump(user_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../data/training_data.pickle', 'wb') as handle:
    pickle.dump(training_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../data/validation_data.pickle', 'wb') as handle:
    pickle.dump(validation_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../data/user_ids.pickle', 'wb') as handle:
    pickle.dump(user_ids, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
del user_matrix
del training_data
del validation_data
del user_ids

In [9]:
import pickle
with open('../data/user_matrix.pickle', 'rb') as handle:
    user_matrix = pickle.load(handle)
with open('../data/training_data.pickle', 'rb') as handle:
    training_data = pickle.load(handle)
with open('../data/validation_data.pickle', 'rb') as handle:
    validation_data = pickle.load(handle)
with open('../data/user_ids.pickle', 'rb') as handle:
    user_ids = pickle.load(handle)

In [10]:
from sklearn.preprocessing import MinMaxScaler

In [11]:
transformer = MinMaxScaler(copy=False)
transformer.fit_transform(user_matrix)
user_matrix

array([[5.0091767e-01, 5.4316151e-01, 6.8304718e-01, ..., 7.9777324e-01,
        3.5412720e-01, 1.1191488e-02],
       [4.9024156e-01, 5.6119668e-01, 6.9976622e-01, ..., 8.4972507e-01,
        6.2900054e-01, 2.9789740e-01],
       [6.5305686e-01, 3.8281393e-01, 6.8315846e-01, ..., 7.9317242e-01,
        3.7172881e-01, 2.3692216e-01],
       ...,
       [5.2600139e-01, 4.7383541e-01, 6.9860792e-01, ..., 8.0544078e-01,
        3.1674346e-01, 3.6427714e-02],
       [5.3288108e-01, 4.9191928e-01, 7.1180719e-01, ..., 7.9153162e-01,
        3.7474704e-01, 8.1391662e-04],
       [5.3903174e-01, 4.4661781e-01, 7.3544085e-01, ..., 7.9921305e-01,
        3.6164027e-01, 1.4010691e-02]], dtype=float32)

In [12]:
k = 50      
def nf_label_transformation_function(label):
    
    one_hot = np.zeros(5)
    one_hot[label - 1] = 1
    return one_hot

import warnings
warnings.simplefilter('ignore')

import keras
#import random
import numpy as np
from keras import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
from generator_helper_functions import  latent_user_input_layer, Generate_User_Latent_Feedforward

#opt = keras.optimizers.rmsprop()
opt = keras.optimizers.SGD()


# k dim user vector + number of movies + statistics + month, day, year
input_size = k + 17770

batch_size = 64

model = Sequential()
model.add(Dense(input_size, activation='relu', input_shape=(input_size,)))
model.add(Dense(1000, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer='sgd',
             metrics=['mse', 'mae', 'acc'])

print("Model Compiled")

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "/home/cody/anaconda3/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/cody/anaconda3/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/cody/anaconda3/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/home/cody/anaconda3/lib/python3.6/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/home/cody/anaconda3/lib/python3.6/imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: libcuda.so.1: cannot open shared object file: No such file or directory


Failed to load the native TensorFlow runtime.

See ht

In [ ]:
def nf_label_transformation_function(label):
    
    one_hot = np.zeros(5)
    one_hot[label - 1] = 1
    return one_hot

train_gen = Generate_User_Latent_Feedforward(training_data, 
                                batch_size, 
                                (input_size,), 
                                (5,), #label shape
                                latent_user_input_layer, 
                                lambda x: nf_label_transformation_function(x[2]),
                                user_matrix,
                                user_ids,
                                normalized = True
                                )

validation_gen = Generate_User_Latent_Feedforward(validation_data, 
                                batch_size, 
                                (input_size,), 
                                (5,), #label shape
                                latent_user_input_layer, 
                                lambda x: nf_label_transformation_function(x[2]),
                                user_matrix,
                                user_ids,
                                normalized = True
                                )

checkpointer = ModelCheckpoint(filepath='models/latent_user_SGD_MSLE_K50_cat_v1__normalized.hdf5', verbose=1, save_best_only=True)
model.fit_generator(train_gen,
                    epochs=5,
                    steps_per_epoch=1000,
                    validation_data=validation_gen,
                    validation_steps=100,
                    callbacks=[checkpointer]
                    )

In [ ]:
validation_data[1]

In [ ]:
from sklearn.metrics import precision_score

In [ ]:
precision_score(
    list(map(lambda x: round(float(model.predict(np.expand_dims(latent_user_input_layer(x, user_matrix, user_ids, True), axis=0)) * 5)), 
             validation_data[0:3000])), 
    validation_data[0:3000,2], average='micro') 



In [ ]:
validation_data[0:3, 2]

In [ ]:
user_ids = {user:i for i, user in enumerate(user_ids)}

In [ ]:
user_rating_matrix

In [ ]:
user_rating_matrix